In [310]:
import itertools
import json
from pathlib import Path

import pandas as pd

In [311]:
def sorted_tuple(x):
    return tuple(sorted(x))

In [312]:
from bellem.dvc.experiment import load_experiments

filepaths = list(Path("../../tmp/drop-mini-manual").glob("*.json"))
experiments = [exp for fp in filepaths for exp in load_experiments(fp)]
print(f"{len(experiments)} experiments")
next(iter(experiments), None)

27 experiments


{'commit': 'workspace',
 'id': 'workspace',
 'name': None,
 'params': {'dataset': {'path': 'bdsaglam/drop-musique-mini',
   'name': 'default',
   'split': 'train'},
  'qa': {'model': 'llama-3-8b',
   'temperature': 0.0,
   'top_p': 0.999,
   'system_prompt': 'direct/format-few.txt',
   'user_prompt_template': 'cq.txt',
   'few_shot_examples': 'manual/direct.json',
   'n_shot': 0,
   'n_sc': 1},
  'run': 1},
 'metrics': {'exact_match': 0.37,
  'f1': 0.5993370884585718,
  'fuzzy_match': 0.41,
  '0hops': {'exact_match': 0.37,
   'f1': 0.5993370884585718,
   'fuzzy_match': 0.41},
  'gen_token_count': {'all': {'count': 100.0,
    'mean': 6.41,
    'std': 4.431510219810265,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 7.0,
    'max': 28.0},
   'success': {'count': 65.0,
    'mean': 5.3076923076923075,
    'std': 2.8718628368789054,
    'min': 3.0,
    '25%': 4.0,
    '50%': 5.0,
    '75%': 5.0,
    'max': 22.0},
   'fail': {'count': 35.0,
    'mean': 8.457142857142857,
    'std

In [313]:
df = pd.json_normalize(experiments)
df.drop(columns=[col for col in df.columns if "fuzzy" in col], inplace=True)
print(f"{len(df)} experiments before preprocessing")
df.head()

27 experiments before preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
0,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.999,direct/format-few.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.00,28.0
1,8a16afb8b9ccefe346f378ef792d57e1d208243d,9ce7ec487e1ad03acf600ba70ecfb9bc147eca9c,nival-care,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,360.00,1857.0,24.0,382.083333,271.094708,160.0,227.0,325.0,374.25,1379.0
2,8a16afb8b9ccefe346f378ef792d57e1d208243d,549da2f4103b6ad971b27bf98e4656589092f610,hexed-vies,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,366.00,1102.0,25.0,383.680000,304.923449,163.0,221.0,280.0,397.00,1333.0
3,8a16afb8b9ccefe346f378ef792d57e1d208243d,c222da6ddcd6bd9282031defcaff5a67350f9cdd,butch-yips,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.950,empty.txt,...,360.25,1303.0,24.0,413.833333,349.656312,141.0,240.0,280.5,377.75,1676.0
4,workspace,workspace,None,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.999,direct/format-few.txt,...,5.00,22.0,35.0,8.457143,5.917784,3.0,5.0,6.0,9.00,28.0


In [314]:
if 'params.qa.top_p' not in df.columns:
    df['params.qa.top_p'] = None

In [315]:
param_cols = [col for col in df.columns if col.startswith("params.")]
metric_cols = [col for col in df.columns if col.startswith("metrics.")]

In [316]:
df.dropna(subset=metric_cols[:2], inplace=True, how="any")
df.drop_duplicates(subset=param_cols, keep='last', inplace=True)

print(f"{len(df)} experiments after preprocessing")
df.head()

23 experiments after preprocessing


,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.success.75%,metrics.gen_token_count.success.max,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max
1,8a16afb8b9ccefe346f378ef792d57e1d208243d,9ce7ec487e1ad03acf600ba70ecfb9bc147eca9c,nival-care,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,360.00,1857.0,24.0,382.083333,271.094708,160.0,227.0,325.0,374.25,1379.0
2,8a16afb8b9ccefe346f378ef792d57e1d208243d,549da2f4103b6ad971b27bf98e4656589092f610,hexed-vies,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,366.00,1102.0,25.0,383.680000,304.923449,163.0,221.0,280.0,397.00,1333.0
3,8a16afb8b9ccefe346f378ef792d57e1d208243d,c222da6ddcd6bd9282031defcaff5a67350f9cdd,butch-yips,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,360.25,1303.0,24.0,413.833333,349.656312,141.0,240.0,280.5,377.75,1676.0
6,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,0e59650eccc512e88827d5222fe221e721ac09fb,owing-gaol,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,1790.00,7334.0,23.0,2044.086957,1310.134202,942.0,1284.0,1570.0,2436.00,6546.0
7,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,67a202554b64a273e95800b8d416dc0f8ca121b7,gemmy-aits,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.95,empty.txt,...,1750.00,5015.0,23.0,2492.347826,1771.907334,1005.0,1366.0,1621.0,2882.50,7196.0


In [317]:
def parse_qa_technique(row):
    model = row['params.qa.model']
    if 'r1' in model:
        base_technique = 'REASONING'
    else:
        base_technique = row["params.qa.system_prompt"].split("/")[0].replace(".txt", "").upper()
    sc = int(row["params.qa.n_sc"])
    sc_suffix = f"-SC({sc})" if sc > 1 else ""
    return f"{base_technique}{sc_suffix}"

df["params.qa.technique"] = df.apply(parse_qa_technique, axis=1)

In [318]:
def remove_ext(filename):
    return filename.split(".", 1)[0]

def parse_instruction(row):
    sp = remove_ext(row["params.qa.system_prompt"]).lower()
    up = remove_ext(row["params.qa.user_prompt_template"]).lower().replace(".txt", "")
    fs = remove_ext(row["params.qa.few_shot_examples"]).lower()
    return f"{sp}:{up}:{fs}"

if len(df):
    df["params.qa.instruction"] = df.apply(parse_instruction, axis=1)

In [319]:
# make temperature 1 decimal
if len(df):
    df["params.qa.temperature"] = df["params.qa.temperature"].round(1)

In [320]:
for col in param_cols:
    values = list(df[col].unique())
    print(f"- {col}: {values}")
    print()

- params.dataset.path: ['bdsaglam/drop-musique-mini']

- params.dataset.name: ['default']

- params.dataset.split: ['train']

- params.qa.model: ['deepseek-r1-llama-70b', 'llama-3-8b', 'llama-3.3-70b']

- params.qa.temperature: [0.5, 0.0]

- params.qa.top_p: [0.95, 0.999, 0.9999]

- params.qa.system_prompt: ['empty.txt', 'direct/format-few.txt', 'direct/format-few-drop.txt', 'ccot/format-thought-drop.txt']

- params.qa.user_prompt_template: ['icq-format-drop.txt', 'cq.txt']

- params.qa.few_shot_examples: ['drop/reasoning.json', 'manual/direct.json', 'drop/direct.json', 'drop/ccot.json']

- params.qa.n_shot: [0]

- params.qa.n_sc: [1, 5]

- params.run: [2, 3, 1]



In [321]:
df.to_json('exps.jsonl', orient='records', lines=True)

## Setup remaining experiments

In [322]:
def produce_experiment_configs(common_params, varying_params):
    # Generate all possible combinations of parameters
    varying_params = {**common_params, **varying_params}
    keys = varying_params.keys()
    values = varying_params.values()
    for instance in itertools.product(*values):
        yield dict(zip(keys, instance))

In [323]:
def produce_all_experiment_configs(common_params: dict, varying_params_list: list[dict]):
    for params in varying_params_list:
        for exp_config in produce_experiment_configs(common_params, params):
            yield exp_config

In [324]:
common_params = {
    "params.dataset.path": ["bdsaglam/drop-musique-mini"],
    "params.dataset.name": ["default"],
    "params.dataset.split": ["train"],
    "params.qa.n_sc": [
        1,
        5,
    ],
    "params.qa.n_shot": [
        0,
        # 1,
    ],
    "params.run": [
        1,
        2,
        3,
    ],
}

In [325]:
varying_params_list = [
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.0
        ],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["direct/format-few-drop.txt"],
        "params.qa.few_shot_examples": ["drop/direct.json"],
    },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.0,
        ],
        "params.qa.top_p": [0.999],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            # "llama-3-8b",
            "llama-3.3-70b",
        ],
        "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["ccot/format-thought-drop.txt"],
        "params.qa.few_shot_examples": ["drop/ccot.json"],
    },
    {
        "params.qa.model": [
            # "deepseek-r1-llama-8b",
            "deepseek-r1-llama-70b",
        ],
         "params.qa.temperature": [
            0.5,
        ],
        "params.qa.top_p": [
            0.95,
        ],
        "params.qa.system_prompt": ["empty.txt"],
        "params.qa.user_prompt_template": ["icq-format-drop.txt"],
        "params.qa.few_shot_examples": ["drop/reasoning.json"],
    },
]

In [326]:
exp_configs = list(produce_all_experiment_configs(common_params, varying_params_list))
target_params = list(exp_configs[0].keys())
print(f"{len(exp_configs)} experiment configurations")
print(target_params)

30 experiment configurations
['params.dataset.path', 'params.dataset.name', 'params.dataset.split', 'params.qa.n_sc', 'params.qa.n_shot', 'params.run', 'params.qa.model', 'params.qa.temperature', 'params.qa.top_p', 'params.qa.system_prompt', 'params.qa.few_shot_examples']


In [327]:
if len(df):
    existing_configs = df[target_params].to_dict(orient="records")
    existing_configs[0]
else:
    existing_configs = []

print("Existing exps:", len(existing_configs))

Existing exps: 23


In [328]:
# find the missing configurations
missing_configs = [
    dict(kv)
    for kv in list(
        {tuple(sorted(config.items())) for config in exp_configs}
        - {tuple(sorted(config.items())) for config in existing_configs}
    )
]
print(f"{len(missing_configs)} missing configurations")

18 missing configurations


In [329]:
def make_command(exp_config):
    lines = ["dvc exp run --queue"]
    for target_param in target_params:
        arg_name = target_param.split(".", 1)[-1]
        arg_value = exp_config[target_param]
        if isinstance(arg_value, str) and '[' in arg_value:
            arg_value = f'"{arg_value}"'
        lines.append(f"-S {arg_name}='{arg_value}'")

    command = " \\\n    ".join(lines)
    return command

In [330]:
with open("run.sh", "w") as f:
    f.write("#!/bin/sh\n\n")
    for exp_config in missing_configs:
        f.write(make_command(exp_config))
        f.write("\n\n")

## Inspect

In [331]:
df

,commit,id,name,params.dataset.path,params.dataset.name,params.dataset.split,params.qa.model,params.qa.temperature,params.qa.top_p,params.qa.system_prompt,...,metrics.gen_token_count.fail.count,metrics.gen_token_count.fail.mean,metrics.gen_token_count.fail.std,metrics.gen_token_count.fail.min,metrics.gen_token_count.fail.25%,metrics.gen_token_count.fail.50%,metrics.gen_token_count.fail.75%,metrics.gen_token_count.fail.max,params.qa.technique,params.qa.instruction
1,8a16afb8b9ccefe346f378ef792d57e1d208243d,9ce7ec487e1ad03acf600ba70ecfb9bc147eca9c,nival-care,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,24.0,382.083333,271.094708,160.0,227.00,325.0,374.25,1379.0,REASONING,empty:icq-format-drop:drop/reasoning
2,8a16afb8b9ccefe346f378ef792d57e1d208243d,549da2f4103b6ad971b27bf98e4656589092f610,hexed-vies,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,25.0,383.680000,304.923449,163.0,221.00,280.0,397.00,1333.0,REASONING,empty:icq-format-drop:drop/reasoning
3,8a16afb8b9ccefe346f378ef792d57e1d208243d,c222da6ddcd6bd9282031defcaff5a67350f9cdd,butch-yips,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,24.0,413.833333,349.656312,141.0,240.00,280.5,377.75,1676.0,REASONING,empty:icq-format-drop:drop/reasoning
6,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,0e59650eccc512e88827d5222fe221e721ac09fb,owing-gaol,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,23.0,2044.086957,1310.134202,942.0,1284.00,1570.0,2436.00,6546.0,REASONING-SC(5),empty:icq-format-drop:drop/reasoning
7,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,67a202554b64a273e95800b8d416dc0f8ca121b7,gemmy-aits,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,23.0,2492.347826,1771.907334,1005.0,1366.00,1621.0,2882.50,7196.0,REASONING-SC(5),empty:icq-format-drop:drop/reasoning
8,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,7ca9f67ec5b0264976ca44c56313dfc29f0b4236,soupy-tuck,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,25.0,2331.120000,2133.850653,1155.0,1346.00,1548.0,1954.00,9171.0,REASONING-SC(5),empty:icq-format-drop:drop/reasoning
9,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,96bac95971d4c717e4e69b1fcd59ef25c70748a3,washy-walk,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,82.0,456.914634,234.460094,162.0,320.25,419.0,504.25,1585.0,REASONING,empty:cq:drop/reasoning
10,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,fcce815e29bd3d0caa201e79fda70e2c8b5c7539,nervy-geum,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,83.0,474.192771,274.875824,194.0,330.50,400.0,498.50,1594.0,REASONING,empty:cq:drop/reasoning
11,e92f3bf0f6fdbcda6bd9e90e6c170a0b265a955a,ee8a52d661a0edf62d235337c0233669e0ffc5ac,wormy-farl,bdsaglam/drop-musique-mini,default,train,deepseek-r1-llama-70b,0.5,0.9500,empty.txt,...,83.0,430.253012,245.710907,191.0,318.50,393.0,474.00,2224.0,REASONING,empty:cq:drop/reasoning
13,c32ca6dbb6474e5546c303462d586427f3672323,c32ca6dbb6474e5546c303462d586427f3672323,main,bdsaglam/drop-musique-mini,default,train,llama-3-8b,0.0,0.9990,direct/format-few.txt,...,35.0,8.457143,5.917784,3.0,5.00,6.0,9.00,28.0,DIRECT,direct/format-few:cq:manual/direct
